# 전처리 ( Preprocessing)

In [1]:
# -*- coding: utf-8 -*-

## Load package

In [2]:
import os
import pandas as pd 
from glob import glob

import sys
import re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
from konlpy.tag import Twitter, Mecab
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1289497, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


## 사용자 사전 추가 및 Stopwords 를 위한 전처리 작업

### 언론사 목록 정리

> stopwords를 위해서

In [3]:
pressList = pd.read_excel('./data/presslist.xlsx')

with(open('./data/newspress.txt','w',encoding='utf-8')) as f:
    f.write('\n'.join(set(pressList['언론사'].tolist())))
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwordList = Stopwords('./data/newspress.txt')

### 공공데이터 포털에서 에서 뉴스 빅데이터 분석 정보로부터 키워드와 명사등의 정보를 추출하기
* 자료 출처 
> * [공공데이터 포털](http://www.data.go.kr/)  
> * [빅카인즈 공공데이터](https://www.kinds.or.kr/)  

In [4]:
path1 = './data/news/high_frequency_noun/'
path3 = './data/news/newstopic/'
path4 = './data/news/people/'
path5 = './data/news/4th_industry/'
path6 = './data/news/have_negative_positive/constitution/'
path7 = './data/news/have_negative_positive/household_debt/'
path8 = './data/news/have_negative_positive/olymphic/'
path9 = './data/news/have_negative_positive/'
path10 = './data/news/etc1/'
path11 = './data/news/etc2/'

### 인물, 기관, 지역, 장소 등의 명사 추출 from news big data in kinds.or.kr

In [7]:
pathList2 = [path11,path10,path4, path5, path6, path7, path8, path9]

In [8]:
def Read_File(file):
    df = pd.read_csv(file,encoding = 'utf-8', engine = 'python')
    targetColumns = list(filter(lambda x: ('인물' in x) or ('기관' in x) or ('지역' in x) or ('장소' in x),df.columns.values))
    if len(targetColumns) != 0:
        return df[targetColumns]
    else:
        return df[targetColumns]
outpath = './data/news/prenoun/'
for idx1, path in enumerate(pathList2):
    fileList = glob(path+'*.csv')
    fileList.extend(glob(path+'CSV'))
    for idx2, file in enumerate(fileList):
        outfile = outpath+str(idx1)+'_'+str(idx2)+'.txt'
        if not os.path.isfile(outfile):
            print (file)
            rawFile = Read_File(file)
            rawFile.to_csv(outfile,index=False, header = False, encoding='utf-8')

./data/news/etc2/김영란법 관련 뉴스메타데이터(20110616-20160731).csv
./data/news/people/연도별 분야별 TOP5 뉴스메이커.csv
./data/news/people/2017년 11월 경제 분야 상위 언급 뉴스 인물 분석.csv
./data/news/people/2017년 11월 정치 분야 상위 언급 뉴스 인물 분석.csv
./data/news/people/2017년 9월 정치 분야 상위 언급 뉴스 인물 분석.csv
./data/news/people/2017년 9월 경제 분야 상위 언급 뉴스 인물 분석.csv
./data/news/people/2017년 10월 정치 분야 상위 언급 뉴스 인물 분석.csv
./data/news/people/2017년 10월 경제 분야 상위 언급 뉴스 인물 분석.csv
./data/news/4th_industry/4차 산업혁명 (20130101-20170630)_1.csv
./data/news/4th_industry/4차 산업혁명 (20130101-20170630)_2.csv
./data/news/4th_industry/4차 산업혁명 (20130101-20170630)_3.csv
./data/news/have_negative_positive/constitution/csv)개헌 관련 뉴스메타데이터(19900101-20161026)_5.csv
./data/news/have_negative_positive/constitution/csv)개헌 관련 뉴스메타데

In [11]:
def Filter1(isList):
    tmp1 = set(isList)
    #tmp3 = set(map(lambda x: x.strip(), set(itertools.chain.from_iterable(tmp2))))
    tmp3 = list(filter(lambda x: len(x)>1, tmp1))
    #tmp3 = list(filter(lambda x: x!='', tmp2))
    #tmp3 = set(map(lambda x: x.strip(), set(itertools.chain.from_iterable(tmp3))))
    tmp3 = list(map(lambda x: x.strip(), tmp3))
    return tmp3
def MakeCombination(li):
    outlist = []
    for ix in range(1, len(li)+1):
        outlist += list(itertools.permutations(li,ix))
    outlist = list(map(lambda x: ' '.join(x), outlist)) + list(map(lambda x: ''.join(x), outlist))
    outlist = list(filter(lambda x: len(x)!=1, outlist))
    outlist = outlist + list(map(lambda x: re.sub('[\W]','', x), outlist))
    return outlist
def filter2(idx):
    idx2 = set(filter(lambda x: x!='', set(MakeCombination(idx.split(' ')))))
    idx2 = set(filter(lambda x: not '  'in x , idx2))
    idx2 = set(map(lambda x: x.strip(), idx2))
    return idx2

In [12]:
path = './data/news/prenoun/'
second_file_list = glob(path+'*.txt')
outList = list()
for file in second_file_list:
    second_file = pd.read_csv(file, encoding = 'utf-8', engine = 'python', header=None)
    for colorder in second_file.columns.values:
        targetData = second_file[colorder]
        targetData.dropna(inplace = True)
        targetData.drop_duplicates(inplace = True)
        targetData.reset_index(drop=True, inplace = True)
        targetData = pd.DataFrame(list(set(list(itertools.chain.from_iterable(targetData.str.split(','))))))
        targetData.reset_index(drop=True, inplace = True)
        targetData.drop_duplicates(inplace = True)
        targetData.reset_index(drop=True, inplace = True)
        targetData = pd.DataFrame(list(set(list(itertools.chain.from_iterable( targetData[0].str.split('[가-힣 ]{3,3}기자'))))))
        targetData = targetData[targetData[0].str.len() >1 ]
        outList += targetData[0].tolist()
outDf = pd.Series(outList)
outDf.dropna(inplace = True)
outDf.drop_duplicates(inplace=True)
outDf.reset_index(drop = True, inplace = True)
tmp1 = outDf[outDf.str.split(' ').str.len() == 1]
#tmp1 = tmp1[tmp1.str.len() <= 15]
tmp1.drop_duplicates(inplace=True)
tmp1.dropna(inplace = True)
tmp1.reset_index(drop = True, inplace = True)
tmp2 = outDf[outDf.str.split(' ').str.len() > 1]
tmp2 = pd.Series(list(set(list(itertools.chain.from_iterable( outDf.str.split(' '))))))
#tmp2 = tmp2[tmp2.str.len() <15]
tmp2.reset_index(drop = True, inplace = True)
tmp2 = tmp2[tmp2.str.len() >1 ]
tmp2.reset_index(drop = True, inplace = True)
tmp2.drop_duplicates(inplace=True)
tmp2.dropna(inplace = True)
tmp1.shape, tmp2.shape

((36409,), (188220,))

In [23]:
outis = pd.concat([tmp1,tmp2])
outis.drop_duplicates(inplace = True)
outis.reset_index(drop = True, inplace = True)
outis = outis[outis.str.match('[a-zA-Z0-9가-힣\s]+$')]
outis.drop_duplicates(inplace = True)
outis.reset_index(drop = True, inplace = True)
print (outis.shape)

(176640,)


### keyword extract from news big data in kinds.or.kr

In [24]:
def Read_File2(file):
    df = pd.read_csv(file,encoding = 'utf-8', engine = 'python')
    targetColumns = list(filter(lambda x: x in ['중요키워드','키워드','keword','토픽키워드','토픽 키워드'], df.columns.values))
    if len(targetColumns) != 0:
        return df[targetColumns]
    else:
        return df[targetColumns]
def Extract_Keywords(path):
    fileList = glob(path+'*.csv')
    fileList.extend(glob(path+'*.CSV'))
    outDf = pd.DataFrame()
    for file in fileList:
        df = Read_File2(file)
        if len(outDf) == 0:
            outDf = df
        else:
            outDf = pd.concat([outDf, df], axis = 0)
    outDf.reset_index(drop=True, inplace = True)
    outDf.astype(str)
    outDf = outDf.to_dict('list')
    outDict = dict()
    for outKey in outDf:
        outDict[outKey] = filter3(outDf[outKey])
    return outDict
def filter3(isList):
    tmp1 = set(isList)
    tmp1 = list(filter(lambda x: type(x)!=float, tmp1))
    tmp2 = list(map(lambda x: x.split(','), tmp1))
    tmp3 = set(map(lambda x: x.strip(), set(itertools.chain.from_iterable(tmp2))))
    tmp3 = list(filter(lambda x: len(x)>1, tmp3))
    tmp3 = list(filter(lambda x: x!='', tmp3))
    tmp3 = list(map(lambda x: x.strip(), tmp3))
    return tmp3

In [25]:
pathList1 = [path3, path4, path5, path6, path7, path8, path9, path10,path11]
#pathList1 = [path1]
#outpath = './data/news/'
outpath = './data/news/keywords/'
outSet2 =set()
for path1 in pathList1:
    ek = Extract_Keywords(path1)
    for info2 in ek:
        out2 = pd.DataFrame(ek[info2])
        out2 = out2[out2[0].str.match('[a-zA-Z0-9가-힣\s]+$')]
        out2.reset_index(drop=True, inplace=True)
        out2 = out2[0].apply(lambda x: filter2(x))
        out2.apply(lambda x: outSet2.update(x))

In [283]:
outis2 = pd.Series(list(outSet2))
outis2.shape
outis2.reset_index(inplace = True, drop=True)
outis2.drop_duplicates(inplace = True)
outis2.shape

(436204,)

In [284]:
beforeFilter = pd.concat([outis,outis2])
beforeFilter.drop_duplicates(inplace = True)
beforeFilter.reset_index(drop=True, inplace = True)
print (beforeFilter.shape)

(515979,)


In [285]:
beforeFilter = beforeFilter[~beforeFilter.str.contains('[a-zA-Z0-9]+')]
beforeFilter.drop_duplicates(inplace = True)
beforeFilter.reset_index(drop=True, inplace = True)
print (beforeFilter.shape)

(464037,)


In [286]:
pickle.dump(beforeFilter,open('./data/pre_data/keywords_rawdata.pickled','wb'))

In [310]:
beforeFilter = pickle.load(open('./data/pre_data/keywords_rawdata.pickled','rb'))

In [311]:
beforeFilter2_ct = beforeFilter.apply(ct.pos)
beforeFilter2_mecab = beforeFilter.apply(mecab.pos)

In [59]:
pickle.dump(beforeFilter2_ct,open('./data/pre_data/keywords_tagging_ct.pickled','wb'))
pickle.dump(beforeFilter2_mecab,open('./data/pre_data/keywords_tagging_mecab.pickled','wb'))

In [312]:
beforeFilter2_ct = pickle.load(open('./data/pre_data/keywords_tagging_ct.pickled','rb'))
beforeFilter2_mecab = pickle.load(open('./data/pre_data/keywords_tagging_mecab.pickled','rb'))

In [371]:
f1 = beforeFilter[((beforeFilter2_ct.str.len() == 1) & (beforeFilter2_mecab.str.len() == 1))]
f1.shape

(77788,)

In [372]:
f1_1noun = f1[(beforeFilter2_mecab[f1.index].apply(lambda x: x[-1][-1] in ['NNP','NNG'])) & (beforeFilter2_ct[f1.index].apply(lambda x: x[-1][-1] == 'Noun'))]
f1_mul_noun = f1[f1.index.difference(f1_1noun.index)]
f1_1noun.shape, f1_mul_noun.shape

((75234,), (2554,))

* 품사 태깅이 하나만 되어 있고, mecab과 twitter 양쪽에서 명사로 판별함 : 75234 -> 사용할 필요 없음
* 그렇지 않은 것 : 2554 -> 버림

In [373]:
f2 = beforeFilter[beforeFilter.index.difference(f1.index)]
print (f2.shape)
f2.drop(f2[f2.str.match('[써씨더서은만랑큼음는인에이게가으로기다을를들지측]+$')].index, inplace = True)
f2.dropna(inplace = True)
f2.drop_duplicates(inplace = True)
print (f2.shape)

(386249,)
(386084,)


In [374]:
f2_mecab = beforeFilter2_mecab[f2.index]
f2_ct = beforeFilter2_ct[f2.index]
print (f2_mecab.shape, f2_ct.shape)
#f2.reset_index(drop=True, inplace = True)
#f2_mecab.reset_index(drop=True, inplace = True)
#f2_ct.reset_index(drop=True, inplace = True)

(386084,) (386084,)


In [375]:
f2_1n_mecab = f2_mecab[(f2_mecab.str.len() == 1) & (f2_mecab.apply(lambda x: x[-1][-1] in ['NNP','NNG']))]
f2_1n_ct = f2_ct[(f2_ct.str.len() == 1) & (f2_ct.apply(lambda x: x[-1][-1] =='Noun'))]
print (f2_1n_mecab.shape, f2_1n_ct.shape)

(83769,) (17683,)


In [376]:
f2_mul_mecab = f2_mecab[f2_mecab.index.difference(f2_1n_mecab.index)]
f2_mul_ct = f2_ct[f2_ct.index.difference(f2_1n_ct.index)]
print (f2_mul_mecab.shape, f2_mul_ct.shape)

(302315,) (368401,)


In [377]:
f2_mul_mecab = f2_mul_mecab[f2_mul_mecab.str.len() != 1]
f2_mul_ct = f2_mul_ct[f2_mul_ct.str.len() != 1]
print (f2_mul_mecab.shape, f2_mul_ct.shape)

(301453,) (367367,)


* 명사가 아닌데 품사 태깅이 하나만 된 것 제거

In [378]:
f2_1n_mecab.shape[0], f2_mul_mecab.shape[0], f2_1n_ct.shape[0], f2_mul_ct.shape[0]

(83769, 301453, 17683, 367367)

In [379]:
f2_mul_mecab.drop(f2_1n_ct.index, inplace = True)
f2_mul_ct.drop(f2_1n_mecab.index, inplace = True)
print (f2_mul_mecab.shape, f2_mul_ct.shape)

(283770,) (283598,)


* mecab과 twitter 둘 중 하나의 품사 태깅에서 품사 태깅이 1개만 되고 명사로 태깅됨

In [387]:
data1is = pd.concat([f2[f2_1n_ct.index], f2[f2_1n_mecab.index]])
data1is.dropna(inplace=True)
data1is.drop_duplicates(inplace = True)
data1is.reset_index(drop = True, inplace =True)
print (data1is.shape)

(101452,)


* mecab과 twitter 둘 다에서 다수의 품사 태깅이 됨  ->> 제거